In [1]:
import numpy as np
import nltk
#nltk.download('punkt')
from bangla_stemmer.stemmer import stemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import json

In [2]:
stmr = stemmer.BanglaStemmer()

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [4]:
def stem(word):
    return stmr.stem(word)

In [5]:
def bag_of_words(tokenized_sentence, words):
    
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [7]:
with open('intents.json', 'r', encoding='utf-8') as f:
    intents = json.load(f)

In [8]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [9]:
# stem and lower each word
ignore_words = ['?', '.', '!',',',';']
# print(all_words)
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort

all_words = sorted(set(all_words))

tags = sorted(set(tags))

In [10]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [37]:
num_epochs = 10000
batch_size = 16
learning_rate = 0.00001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

317 13


In [38]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [39]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [40]:
device

device(type='cuda')

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')
print(f'Accuracy: {100.00-loss.item()*100}')

Epoch [100/10000], Loss: 1.3108
Epoch [200/10000], Loss: 0.1727
Epoch [300/10000], Loss: 0.0872
Epoch [400/10000], Loss: 0.2825
Epoch [500/10000], Loss: 0.9183
Epoch [600/10000], Loss: 0.1734
Epoch [700/10000], Loss: 0.2486
Epoch [800/10000], Loss: 1.0138
Epoch [900/10000], Loss: 0.0518
Epoch [1000/10000], Loss: 0.0863
Epoch [1100/10000], Loss: 0.2748
Epoch [1200/10000], Loss: 0.1954
Epoch [1300/10000], Loss: 0.2666
Epoch [1400/10000], Loss: 1.1055
Epoch [1500/10000], Loss: 0.1616
Epoch [1600/10000], Loss: 0.2321
Epoch [1700/10000], Loss: 0.1964
Epoch [1800/10000], Loss: 0.0099
Epoch [1900/10000], Loss: 0.0586
Epoch [2000/10000], Loss: 0.0833
Epoch [2100/10000], Loss: 0.5454
Epoch [2200/10000], Loss: 0.5909
Epoch [2300/10000], Loss: 0.1243
Epoch [2400/10000], Loss: 0.0344
Epoch [2500/10000], Loss: 0.0400
Epoch [2600/10000], Loss: 0.1425
Epoch [2700/10000], Loss: 0.0226
Epoch [2800/10000], Loss: 0.0497
Epoch [2900/10000], Loss: 0.1419
Epoch [3000/10000], Loss: 0.0937
Epoch [3100/10000],

In [46]:
model_parameters = model.state_dict()

In [58]:
model.eval()

NeuralNet(
  (l1): Linear(in_features=317, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=13, bias=True)
  (relu): ReLU()
)

In [45]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
